# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/rce_pilot_2_00_trodes_metadata.pkl")

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,"[980, 1181]","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,"[982229, 1182226]","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,20230612_101430_standard_comp_to_training_D1_s...,"[3376, 3575]","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,"[3382227, 3582224]","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,20230612_101430_standard_comp_to_training_D1_s...,"[5672, 5871]","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,"[5682225, 5882222]","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,20230612_101430_standard_comp_to_training_D1_s...,"[7468, 7668]","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,"[7482224, 7682221]","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,20230612_101430_standard_comp_to_training_D1_s...,"[8566, 8765]","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,"[8582220, 8782223]","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
...,...,...,...,...,...,...,...,...,...,...,...
58,20230630_115506_standard_comp_to_novel_agent_D...,"[60714, 60914]","[[618, 655], [1130, 1140], [1246, 1444], [1465...","[[939, 941], [1353, 1353], [1354, 1368], [1375...",20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,"[60673457, 60873459]","[[619332, 654132], [1131540, 1141541], [124834...","[[940936, 942136], [1354140, 1355143], [135654..."
58,20230630_115506_standard_comp_to_novel_agent_D...,"[62211, 62410]","[[618, 655], [1130, 1140], [1246, 1444], [1465...","[[939, 941], [1353, 1353], [1354, 1368], [1375...",20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,"[62173475, 62373477]","[[619332, 654132], [1131540, 1141541], [124834...","[[940936, 942136], [1354140, 1355143], [135654..."
58,20230630_115506_standard_comp_to_novel_agent_D...,"[64207, 64406]","[[618, 655], [1130, 1140], [1246, 1444], [1465...","[[939, 941], [1353, 1353], [1354, 1368], [1375...",20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,"[64173502, 64373501]","[[619332, 654132], [1131540, 1141541], [124834...","[[940936, 942136], [1354140, 1355143], [135654..."
58,20230630_115506_standard_comp_to_novel_agent_D...,"[66104, 66303]","[[618, 655], [1130, 1140], [1246, 1444], [1465...","[[939, 941], [1353, 1353], [1354, 1368], [1375...",20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,"[66073522, 66273524]","[[619332, 654132], [1131540, 1141541], [124834...","[[940936, 942136], [1354140, 1355143], [135654..."


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/rce_pilot_2_per_video_trial_labels.pkl")

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,980,1181,[1028 1031] [1149 1266],,20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,982229,1182226,[1030229 1033226] [1151634 1269428],
0,20230612_101430_standard_comp_to_training_D1_s...,3376,3575,[3545 3545] [3547 3549] [3550 3554] [3554 3556...,,20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,3382227,3582224,[3550827 3551624] [3553824 3555829] [3556426 3...,
0,20230612_101430_standard_comp_to_training_D1_s...,5672,5871,[5761 5762] [5762 5942],,20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,5682225,5882222,[5771223 5772822] [5773422 5952622],
0,20230612_101430_standard_comp_to_training_D1_s...,7468,7668,[7632 7634] [7635 7665] [7665 7945],,20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,7482224,7682221,[7647221 7648224] [7649024 7679421] [7680023 7...,
0,20230612_101430_standard_comp_to_training_D1_s...,8566,8765,[8678 8733] [8735 8975],,20230612_101430_standard_comp_to_training_D1_s...,"[1.3, 1.4]",8798886,77093151,8582220,8782223,[8694220 8750020] [8750420 8992222],


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
58,20230630_115506_standard_comp_to_novel_agent_D...,60714,60914,[60676 60746] [60777 60803] [60824 60897] [609...,[60819 60821] [60821 61167],20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,60673457,60873459,[60635256 60705256] [60736657 60762662] [60782...,[60779058 60780258] [60780662 61127062]
58,20230630_115506_standard_comp_to_novel_agent_D...,62211,62410,[62160 62396],[62226 62228] [62230 62236] [62260 62283] [622...,20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,62173475,62373477,[62121874 62359482],[62189072 62190075] [62193075 62198472] [62221...
58,20230630_115506_standard_comp_to_novel_agent_D...,64207,64406,[64188 64391] [64393 64738],[64298 64477],20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,64173502,64373501,[64154499 64357501] [64359306 64705110],[64264904 64443702]
58,20230630_115506_standard_comp_to_novel_agent_D...,66104,66303,[66124 66274] [66275 66276] [66278 66523],[66153 66163] [66256 66257] [66257 66319],20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,66073522,66273524,[66093722 66244528] [66244928 66246724] [66248...,[66122925 66133522] [66226128 66227328] [66227...
58,20230630_115506_standard_comp_to_novel_agent_D...,67102,67301,[67115 67213] [67231 67606],[67116 67144] [67158 67164] [67199 67498],20230630_115506_standard_comp_to_novel_agent_D...,"[1.2, 1.4, 2.1, 2.2]",10971930,79536439,67073535,67273537,[67086534 67184943] [67202335 67578740],[67087934 67115334] [67129134 67135935] [67170...


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/rce_pilot_2_per_video_trial_labels.csv")